In [2]:
import pandas as pd
import numpy as np 
from glob import glob
import cv2
from skimage import io 
from tqdm import tqdm
import seaborn as sns
from tensorflow import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory, image
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D, InputLayer
from tensorflow.keras import layers
import tensorflow.keras.optimizers as Optimizer
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.layers import (
    Input, Dense, Conv2D, Flatten, Activation, 
    MaxPooling2D, AveragePooling2D, ZeroPadding2D, GlobalAveragePooling2D, GlobalMaxPooling2D, add
)
import pandas as pd
import numpy as np 
from glob import glob
import cv2
from skimage import io 
from tqdm import tqdm
import seaborn as sns
from tensorflow import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory, image
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D, InputLayer
from tensorflow.keras import layers
import tensorflow.keras.optimizers as Optimizer
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping()
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.utils import plot_model

from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications.inception_v3 import InceptionV3
#from tensorflow.keras.applications.inception_v3 import preprocess_input

In [15]:
ground_truth_df = pd.read_csv("ISIC_2019_Training_GroundTruth.csv")
meta_data_df = pd.read_csv("ISIC_2019_Training_Metadata.csv")
ground_truth_df

,image,MEL,NV,BCC,AK,BKL,DF,VASC,SCC,UNK
0,ISIC_0000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ISIC_0000001,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ISIC_0000002,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ISIC_0000003,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ISIC_0000004,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
25326,ISIC_0073247,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
25327,ISIC_0073248,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
25328,ISIC_0073249,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25329,ISIC_0073251,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
cols=['NV','BCC','AK','BKL','DF','VASC','SCC','MEL', "UNK"]


In [17]:
for i in range(0, len(cols)):
    print("Total images of " + cols[i] + " : " + str(len(ground_truth_df[ground_truth_df[cols[i]]>0])))

Total images of NV : 12875
Total images of BCC : 3323
Total images of AK : 867
Total images of BKL : 2624
Total images of DF : 239
Total images of VASC : 253
Total images of SCC : 628
Total images of MEL : 4522
Total images of UNK : 0


In [13]:
meta_data_df

,image,age_approx,anatom_site_general,lesion_id,sex
0,ISIC_0000000,55.0,anterior torso,NaN,female
1,ISIC_0000001,30.0,anterior torso,NaN,female
2,ISIC_0000002,60.0,upper extremity,NaN,female
3,ISIC_0000003,30.0,upper extremity,NaN,male
4,ISIC_0000004,80.0,posterior torso,NaN,male
...,...,...,...,...,...
25326,ISIC_0073247,85.0,head/neck,BCN_0003925,female
25327,ISIC_0073248,65.0,anterior torso,BCN_0001819,male
25328,ISIC_0073249,70.0,lower extremity,BCN_0001085,male
25329,ISIC_0073251,55.0,palms/soles,BCN_0002083,female


In [18]:
data_df= meta_data_df.merge(ground_truth_df, how= 'inner', on='image').sample(frac=1).reset_index(drop=True) #frac keyword speicifies the fraction of rows to return in the random sample so frac =1 meants to retuen all rows in random order
data_df

,image,age_approx,anatom_site_general,lesion_id,sex,MEL,NV,BCC,AK,BKL,DF,VASC,SCC,UNK
0,ISIC_0028244,50.0,lower extremity,HAM_0001533,male,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ISIC_0026621,45.0,anterior torso,HAM_0001494,female,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ISIC_0056055,30.0,anterior torso,BCN_0000099,male,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ISIC_0000251,50.0,posterior torso,NaN,female,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ISIC_0060193,55.0,head/neck,BCN_0002963,male,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25326,ISIC_0055940,80.0,anterior torso,BCN_0001075,male,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
25327,ISIC_0062750,55.0,lower extremity,BCN_0004304,female,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25328,ISIC_0059162,80.0,anterior torso,BCN_0004428,male,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
25329,ISIC_0068863,35.0,head/neck,BCN_0005007,female,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
#The batch size defines the number of samples that will be propagated through the network.

#For instance, let's say you have 1050 training samples and you want to set up a batch_size equal to 100. 
#The algorithm takes the first 100 samples (from 1st to 100th) from the training dataset and trains the network.
#Next, it takes the second 100 samples (from 101st to 200th) and trains the network again. 
#We can keep doing this procedure until we have propagated all samples through of the network. 
#Problem might happen with the last set of samples. In our example, 


CFG = dict(
        batch_size        =  16,     # 8; 16; 32; 64; bigger batch size => moemry allocation issue
        epochs            =  100,   # 5; 10; 20;
        verbose           =   1,    # 0; 1 verbose=0 will show you nothing (silent), verbose=1 will show you an animated progress bar like this:progres_bar, verbose=2 will just mention the number of epoch
        workers           =   4,    # 1; 2; 3  number of threads generating batches in parallel. Batches are computed in parallel on the CPU and passed on the fly onto the GPU for neural network computations

        optimizer         = 'adam', # 'SGD', 'RMSprop'
        RANDOM_STATE      =  123,   
    
        # Path to save a model
        path_model        = '../working/',

        # Images sizes
        img_size          = 224, 
        img_height        = 224, 
        img_width         = 224, 

        # Images augs
        ROTATION          = 180.0,
        ZOOM              =  10.0,
        ZOOM_RANGE        =  [0.9,1.1],
        HZOOM             =  10.0,
        WZOOM             =  10.0,
        HSHIFT            =  10.0,
        WSHIFT            =  10.0,
        SHEAR             =   5.0,
        HFLIP             = True,
        VFLIP             = True,

        # Postprocessing
        label_smooth_fac  =  0.00,  # 0.01; 0.05; 0.1; 0.2;    
)

In [ ]:
training = image_dataset_from_directory("Image_dir_2",
                                       subset= "training",
                                       validation_split=.2,
                                       seed=10)
    
validation = image_dataset_from_directory("Image_dir_2",
                                       subset= "validation",
                                       validation_split=.2,
                                       seed=10)

#set up set of folders for training and validation 

In [11]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.3,
    rotation_range            = CFG['ROTATION'],
    zoom_range                = CFG['ZOOM_RANGE'],  #is for randomly zooming inside pictures  
    horizontal_flip           = CFG['HFLIP'], #is for randomly flipping half of the images horizontally --relevant when there are no assumptions of horizontal assymetry (e.g. real-world pictures).
    vertical_flip             = CFG['VFLIP'],
    height_shift_range        = CFG['HSHIFT'],
    width_shift_range         = CFG['WSHIFT'],
    shear_range               = CFG['SHEAR'], #randomly applying shearing transformations
    channel_shift_range       = 0.0,
    brightness_range          = None,
    fill_mode                 = 'nearest',      #is the strategy used for filling in newly created pixels, which can appear after a rotation or a width/height shift                     
    )

valid_generator = ImageDataGenerator(rescale=1./255, validation_split=0.3)              # no aug for valid
test_generator  = ImageDataGenerator(rescale=1./255)                                    # no aug for test
# Train data
train_generator = train_datagen.flow_from_directory("Image_dir_2",
                                                    subset='training',                  # to read train/valid from same directory 
                                                    target_size=(CFG['img_size'], CFG['img_size']),
                                                    batch_size = CFG['batch_size'],
                                                    class_mode='categorical',
                                                    )

# Validation data
valid_generator = valid_generator.flow_from_directory("Image_dir_2",
                                                     subset='validation',               # to read train/valid from same directory 
                                                     target_size=(CFG['img_size'], CFG['img_size']),
                                                     batch_size = CFG['batch_size'],
                                                     class_mode='categorical'
                                                     ) 
# Test data
test_generator  = test_generator.flow_from_directory("Image_dir_2",
                                                     target_size=(CFG['img_size'], CFG['img_size']),
                                                     batch_size = 1,                    # using 1 to easily manage mapping between test_gen & pred
                                                     class_mode='categorical'
                                                     )

Found 17736 images belonging to 8 classes.
Found 7595 images belonging to 8 classes.
Found 25331 images belonging to 8 classes.


In [41]:
model_ResNet50 = tf.keras.Sequential([
     tf.keras.applications.ResNet50(
        input_shape=(224, 224, 3),
        weights='imagenet',
        include_top=False
    ),
    
    GlobalAveragePooling2D(),
    
    #Dense(1024, activation = 'relu'), 
    #Dropout(0.5), 
    #BatchNormalization(),
    
    #Dense(256, activation='relu'), 
    #Dropout(0.3), 
    #BatchNormalization(),
    
    #Dense(64, activation='relu'), 
    #Dropout(0.2), 
    #BatchNormalization(),
    
    Dense(8, activation='softmax') # num classes = 9
    
])
    
model_ResNet50.compile(
    optimizer = CFG['optimizer'],
    loss = tf.keras.losses.BinaryCrossentropy(label_smoothing = CFG['label_smooth_fac']),
    #loss = 'binary_crossentropy',
    metrics=['accuracy']
)

In [44]:
#tf.function-decorated function tried to create variables on non-first call'. 
tf.config.run_functions_eagerly(True) # otherwise error

# https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint
cb_early_stopper = EarlyStopping(monitor = 'val_loss', patience = 10)
cb_checkpointer  = ModelCheckpoint(#filepath=path_model,
                                   #filepath=CFG['path_model']+'ResNet50.hdf5'
                                   filepath = CFG['path_model']+'ResNet50-{epoch:02d}-{val_loss:.2f}.hdf5',
                                   monitor  = 'val_loss', 
                                   verbose  = CFG['verbose'], 
                                   save_best_only=True, 
                                   mode='min'
                                  )

callbacks_list = [cb_checkpointer, cb_early_stopper]

history3 = model_ResNet50.fit(train_generator, 
                             epochs=CFG['epochs'], 
                             workers=CFG['workers'],
                             #steps_per_epoch = train_generator.n // 2, # hide if you wish
                             validation_data=valid_generator, 
                             #validation_steps=valid_generator.n // 2,  # hide if you wish
                             callbacks = callbacks_list,
                            )

/Users/snizhanakurylyuk/opt/anaconda3/envs/metis/lib/python3.7/site-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


Epoch 1/100
1109/1109 [==============================] - ETA: 0s - loss: 0.2295 - accuracy: 0.5985
Epoch 00001: val_loss improved from inf to 0.31380, saving model to ../working/ResNet50-01-0.31.hdf5
1109/1109 [==============================] - 6548s 6s/step - loss: 0.2295 - accuracy: 0.5985 - val_loss: 0.3138 - val_accuracy: 0.4686
Epoch 2/100
1109/1109 [==============================] - ETA: 0s - loss: 0.2046 - accuracy: 0.6411
Epoch 00002: val_loss did not improve from 0.31380
1109/1109 [==============================] - 6977s 6s/step - loss: 0.2046 - accuracy: 0.6411 - val_loss: 0.7553 - val_accuracy: 0.5111
Epoch 3/100
1109/1109 [==============================] - ETA: 0s - loss: 0.1959 - accuracy: 0.6567
Epoch 00003: val_loss did not improve from 0.31380
1109/1109 [==============================] - 6998s 6s/step - loss: 0.1959 - accuracy: 0.6567 - val_loss: 0.4235 - val_accuracy: 0.5232
Epoch 4/100
1109/1109 [==============================] - ETA: 0s - loss: 0.1899 - accuracy: 0.6

In [ ]:
model_ResNet50.evaluate(train_generator)

  28/1109 [..............................] - ETA: 34:41 - loss: 0.1357 - accuracy: 0.7679

In [5]:
CNN= Sequential()
CNN.add(layers.InputLayer(input_shape= (224, 224, 3)))

#each block increases model capacity 

CNN.add(Conv2D(filters=10, kernel_size=3, activation ="relu", padding = "same"))
CNN.add(Conv2D(filters=20, kernel_size=3, activation ="relu", padding = "same"))
CNN.add(MaxPooling2D())

#The conv blocks shouls be ended with either a flatten 
#layer or a global pooling layer. These transform the 2D layers to 1D
#to match the following dense layer
CNN.add(Conv2D(filters=30, kernel_size=3, activation ="relu", padding = "same"))
CNN.add(GlobalAveragePooling2D())

#fully connected block- flattening followed by dense and outputlayers

CNN.add(Dense(20, activation= 'relu'))
CNN.add(Dense(8, activation = 'softmax')) #most likely will need an 8 
CNN.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 224, 224, 10)      280       
                                                                 
 conv2d_4 (Conv2D)           (None, 224, 224, 20)      1820      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 112, 112, 20)     0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 112, 112, 30)      5430      
                                                                 
 global_average_pooling2d_1   (None, 30)               0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_2 (Dense)             (None, 20)               

In [9]:
CNN.compile(CFG['optimizer'],
           loss= "categorical_crossentropy",
           metrics= ["accuracy"])

In [15]:
#tf.function-decorated function tried to create variables on non-first call'. 
tf.config.run_functions_eagerly(True) # otherwise error

# https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint
cb_early_stopper = EarlyStopping(monitor = 'val_loss', patience = 10)
cb_checkpointer  = ModelCheckpoint(#filepath=path_model,
                                   #filepath=CFG['path_model']+'ResNet50.hdf5'
                                   filepath = CFG['path_model']+'ResNet50-{epoch:02d}-{val_loss:.2f}.hdf5',
                                   monitor  = 'val_loss', 
                                   verbose  = CFG['verbose'], 
                                   save_best_only=True, 
                                   mode='min'
                                  )

callbacks_list = [cb_checkpointer, cb_early_stopper]

history2 = CNN.fit(train_generator, 
                             epochs=2, 
                             workers=CFG['workers'],
                             validation_data=valid_generator, 
                             callbacks = callbacks_list,
                            )

Epoch 1/2
   3/1109 [..............................] - ETA: 44:22 - loss: 1.5398 - accuracy: 0.4583 

KeyboardInterrupt: 

# CNN Model 2

In [24]:
import tensorflow.keras.models as Models
model = Models.Sequential()

model.add(layers.Conv2D(200,kernel_size=(3,3),activation='relu',input_shape=(224,224,3)))
model.add(layers.Conv2D(180,kernel_size=(3,3),activation='relu'))
model.add(layers.MaxPool2D(5,5))
model.add(layers.Conv2D(180,kernel_size=(3,3),activation='relu'))
model.add(layers.Conv2D(140,kernel_size=(3,3),activation='relu'))
model.add(layers.Conv2D(100,kernel_size=(3,3),activation='relu'))
model.add(layers.Conv2D(50,kernel_size=(3,3),activation='relu'))
model.add(layers.MaxPool2D(5,5))
model.add(layers.Flatten())
model.add(layers.Dense(180,activation='relu'))
model.add(layers.Dense(100,activation='relu'))
model.add(layers.Dense(50,activation='relu'))
model.add(layers.Dropout(rate=0.5))
model.add(layers.Dense(8,activation='softmax'))



In [25]:
model.compile(
    optimizer = CFG['optimizer'],
    loss = "categorical_crossentropy",
    metrics=['accuracy']
)

In [26]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 222, 222, 200)     5600      
                                                                 
 conv2d_19 (Conv2D)          (None, 220, 220, 180)     324180    
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 44, 44, 180)      0         
 2D)                                                             
                                                                 
 conv2d_20 (Conv2D)          (None, 42, 42, 180)       291780    
                                                                 
 conv2d_21 (Conv2D)          (None, 40, 40, 140)       226940    
                                                                 
 conv2d_22 (Conv2D)          (None, 38, 38, 100)       126100    
                                                      

In [29]:
#tf.function-decorated function tried to create variables on non-first call'. 
tf.config.run_functions_eagerly(True) # otherwise error

# https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint
cb_early_stopper = EarlyStopping(monitor = 'val_loss', patience = 10)
cb_checkpointer  = ModelCheckpoint(filepath = CFG['path_model']+'CNN-{epoch:02d}-{val_loss:.2f}.hdf5',
                                   monitor  = 'val_loss', 
                                   verbose  = CFG['verbose'], 
                                   save_best_only=True, 
                                   mode='min'
                                  )

callbacks_list = [cb_checkpointer, cb_early_stopper]

history3 = model.fit(train_generator, 
                             epochs=2, 
                             workers=CFG['workers'],
                             validation_data=valid_generator, 
                             callbacks = callbacks_list,
                            )

Epoch 1/2
1109/1109 [==============================] - ETA: 0s - loss: 1.3893 - accuracy: 0.5148 
Epoch 00001: val_loss improved from inf to 1.30128, saving model to ../working/CNN-01-1.30.hdf5
1109/1109 [==============================] - 38106s 34s/step - loss: 1.3893 - accuracy: 0.5148 - val_loss: 1.3013 - val_accuracy: 0.5368
Epoch 2/2
1109/1109 [==============================] - ETA: 0s - loss: 1.2833 - accuracy: 0.5415 
Epoch 00002: val_loss improved from 1.30128 to 1.29645, saving model to ../working/CNN-02-1.30.hdf5
1109/1109 [==============================] - 28602s 26s/step - loss: 1.2833 - accuracy: 0.5415 - val_loss: 1.2964 - val_accuracy: 0.5315


# CNN Model 3

In [34]:
CFG2 = dict(
        batch_size        =  32,     # 8; 16; 32; 64; bigger batch size => moemry allocation issue
        epochs            =  100,   # 5; 10; 20;
        verbose           =   1,    # 0; 1
        workers           =   4,    # 1; 2; 3
        optimizer         = 'adam', # 'SGD', 'RMSprop'

        RANDOM_STATE      =  123,   
    
        # Path to save a model
        path_model        = '../working/',

        # Images sizes
        img_size          = 224, 
        img_height        = 224, 
        img_width         = 224, 

        # Images augs
        ROTATION          = 180.0,
        ZOOM              =  10.0,
        ZOOM_RANGE        =  [0.9,1.1],
        HZOOM             =  8.0, #maybe try 8.0?
        WZOOM             =  8.0, #maybe try 8.0?
        HSHIFT            =  8.0, #maybe try 8.0?
        WSHIFT            =  8.0, #maybe try 8.0?
        SHEAR             =   2.0, #maybe try 2.0?
        HFLIP             = True,
        VFLIP             = True,

        # Postprocessing
        label_smooth_fac  =  0.05,  # 0.01; 0.05; 0.1; 0.2;     Label smoothing is used when the loss function is cross entropy, and the model applies the softmax function to the penultimate layer’s logit vectors z to compute its output probabilities p. 
)

#CHANGED ZOOM, SHift, smooth_fac and Batchsize

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.3, #Rescale 1./255 is to transform every pixel value from range [0,255] -> [0,1]. And the benefits are: Treat all images in the same manner: some images are high pixel range, some are low pixel range. 
                                   #The images are all sharing the same model, weights and learning rate.
    rotation_range            = CFG2['ROTATION'],
    zoom_range                = CFG2['ZOOM_RANGE'],  #is for randomly zooming inside pictures  
    horizontal_flip           = CFG2['HFLIP'], #is for randomly flipping half of the images horizontally --relevant when there are no assumptions of horizontal assymetry (e.g. real-world pictures).
    vertical_flip             = CFG2['VFLIP'],
    height_shift_range        = CFG2['HSHIFT'],
    width_shift_range         = CFG2['WSHIFT'],
    shear_range               = CFG2['SHEAR'], #randomly applying shearing transformations
    channel_shift_range       = 0.0,
    brightness_range          = None,
    fill_mode                 = 'nearest',      #is the strategy used for filling in newly created pixels, which can appear after a rotation or a width/height shift                     
    )

valid_generator = ImageDataGenerator(rescale=1./255, validation_split=0.3)              # no aug for valid
test_generator  = ImageDataGenerator(rescale=1./255)                                    # no aug for test
# Train data
train_generator = train_datagen.flow_from_directory("Image_dir_2",
                                                    subset='training',                  # to read train/valid from same directory 
                                                    target_size=(CFG2['img_size'], CFG2['img_size']),
                                                    batch_size = CFG2['batch_size'],
                                                    class_mode='categorical',
                                                    )

# Validation data
valid_generator = valid_generator.flow_from_directory("Image_dir_2",
                                                     subset='validation',               # to read train/valid from same directory 
                                                     target_size=(CFG2['img_size'], CFG2['img_size']),
                                                     batch_size = CFG2['batch_size'],
                                                     class_mode='categorical'
                                                     ) 
# Test data
test_generator  = test_generator.flow_from_directory("Image_dir_2",
                                                     target_size=(CFG2['img_size'], CFG2['img_size']),
                                                     batch_size = 1,                    # using 1 to easily manage mapping between test_gen & pred
                                                     class_mode='categorical'
                                                     )

In [30]:
import tensorflow.keras.models as Models
model = Models.Sequential()

model.add(layers.Conv2D(200,kernel_size=(3,3),activation='relu',input_shape=(224,224,3)))
model.add(layers.Conv2D(180,kernel_size=(3,3),activation='relu'))
model.add(layers.MaxPool2D(5,5))
model.add(layers.Conv2D(180,kernel_size=(3,3),activation='relu'))
model.add(layers.Conv2D(140,kernel_size=(3,3),activation='relu'))
model.add(layers.Conv2D(100,kernel_size=(3,3),activation='relu'))
model.add(layers.Conv2D(50,kernel_size=(3,3),activation='relu'))
model.add(layers.MaxPool2D(5,5))
model.add(layers.Flatten())
model.add(layers.Dense(180,activation='relu'))
model.add(layers.Dense(100,activation='relu'))
model.add(layers.Dense(50,activation='relu'))
model.add(layers.Dropout(rate=0.5))
model.add(layers.Dense(8,activation='softmax'))


In [31]:
model.compile(
    optimizer = CFG2['optimizer'],
    loss = "categorical_crossentropy",
    metrics=['accuracy']
)

In [33]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 222, 222, 200)     5600      
                                                                 
 conv2d_25 (Conv2D)          (None, 220, 220, 180)     324180    
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 44, 44, 180)      0         
 2D)                                                             
                                                                 
 conv2d_26 (Conv2D)          (None, 42, 42, 180)       291780    
                                                                 
 conv2d_27 (Conv2D)          (None, 40, 40, 140)       226940    
                                                                 
 conv2d_28 (Conv2D)          (None, 38, 38, 100)       126100    
                                                      

In [36]:
#tf.function-decorated function tried to create variables on non-first call'. 
tf.config.run_functions_eagerly(True) # otherwise error

cb_early_stopper = EarlyStopping(monitor = 'val_loss', patience = 10)
cb_checkpointer  = ModelCheckpoint(filepath = CFG['path_model']+'CNN-{epoch:02d}-{val_loss:.2f}.hdf5',
                                   monitor  = 'val_loss', 
                                   verbose  = CFG['verbose'], 
                                   save_best_only=True, 
                                   mode='min'
                                  )

callbacks_list = [cb_checkpointer, cb_early_stopper]

history3 = model.fit(train_generator, 
                             epochs=2, 
                             workers=CFG2['workers'],
                             validation_data=valid_generator, 
                             callbacks = callbacks_list,
                            )

Epoch 1/2
1109/1109 [==============================] - ETA: 0s - loss: 1.4099 - accuracy: 0.5083 
Epoch 00001: val_loss improved from inf to 1.38955, saving model to ../working/CNN-01-1.39.hdf5
1109/1109 [==============================] - 37603s 34s/step - loss: 1.4099 - accuracy: 0.5083 - val_loss: 1.3896 - val_accuracy: 0.5234
Epoch 2/2
1109/1109 [==============================] - ETA: 0s - loss: 1.2814 - accuracy: 0.5488 
Epoch 00002: val_loss improved from 1.38955 to 1.34985, saving model to ../working/CNN-02-1.35.hdf5
1109/1109 [==============================] - 22595s 20s/step - loss: 1.2814 - accuracy: 0.5488 - val_loss: 1.3499 - val_accuracy: 0.5423


In [40]:
from sklearn.utils import class_weight
from collections import Counter
class_weights= class_weight.compute_class_weight(class_weight= "balanced",
                                    classes= np.unique(train_generator.classes),
                                    y=train_generator.classes)

unique_class_weights = np.unique(train_generator.classes)
class_weights_dict={unique_class_weights[i]: w for i,w in enumerate(class_weights)}

print('\nCLASS WEIGHTS: {}\n'.format(class_weights))
print(np.unique(train_generator.classes))
print(train_generator.classes)
print(unique_class_weights)
print(Counter(train_generator.classes).keys())   # equals to list(set(x))
print(Counter(train_generator.classes).values()) # counts the elements' frequency


CLASS WEIGHTS: [ 3.6523888   0.95272884  1.20685901 13.19642857  0.70025268  0.24597803
  5.03863636 12.45505618]

[0 1 2 3 4 5 6 7]
[0 0 0 ... 7 7 7]
[0 1 2 3 4 5 6 7]
dict_keys([0, 1, 2, 3, 4, 5, 6, 7])
dict_values([607, 2327, 1837, 168, 3166, 9013, 440, 178])


In [41]:
#tf.function-decorated function tried to create variables on non-first call'. 
tf.config.run_functions_eagerly(True) # otherwise error

cb_early_stopper = EarlyStopping(monitor = 'val_loss', patience = 10)
cb_checkpointer  = ModelCheckpoint(filepath = CFG['path_model']+'CNN-{epoch:02d}-{val_loss:.2f}.hdf5',
                                   monitor  = 'val_loss', 
                                   verbose  = CFG['verbose'], 
                                   save_best_only=True, 
                                   mode='min'
                                  )

callbacks_list = [cb_checkpointer, cb_early_stopper]

history3 = model.fit(train_generator, 
                             epochs=CFG2['epochs'], 
                             workers=CFG2['workers'],
                             validation_data=valid_generator, 
                             callbacks = callbacks_list,
                             class_weight = class_weights_dict
                            )

/Users/snizhanakurylyuk/opt/anaconda3/envs/metis/lib/python3.7/site-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


Epoch 1/100
1109/1109 [==============================] - ETA: 0s - loss: 2.0714 - accuracy: 0.3695 
Epoch 00001: val_loss improved from inf to 1.93530, saving model to ../working/CNN-01-1.94.hdf5
1109/1109 [==============================] - 53484s 48s/step - loss: 2.0714 - accuracy: 0.3695 - val_loss: 1.9353 - val_accuracy: 0.4242
Epoch 2/100
1109/1109 [==============================] - ETA: 0s - loss: 2.0850 - accuracy: 0.5081 
Epoch 00002: val_loss did not improve from 1.93530
1109/1109 [==============================] - 37157s 33s/step - loss: 2.0850 - accuracy: 0.5081 - val_loss: 2.0368 - val_accuracy: 0.5085
Epoch 3/100
1109/1109 [==============================] - ETA: 0s - loss: 2.0807 - accuracy: 0.2801 
Epoch 00003: val_loss did not improve from 1.93530
1109/1109 [==============================] - 62499s 56s/step - loss: 2.0807 - accuracy: 0.2801 - val_loss: 2.0635 - val_accuracy: 0.1036
Epoch 4/100
 723/1109 [==================>...........] - ETA: 3:59:51 - loss: 2.0933 - accu

KeyboardInterrupt: 